In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
from langchain.llms import BaseLLM
from langchain.prompts import PromptTemplate
from typing import Dict, List, Tuple

# First, let's create our LegalExpertAgent class that will handle individual agent analysis
class LegalExpertAgent:
    def __init__(self, llm: BaseLLM, stance: str, name: str):
        """Initialize a legal expert agent with a specific stance on case analysis.
        
        Args:
            llm: The language model to use
            stance: Either "advocate" or "critic" to determine the agent's position
            name: Name identifier for the agent
        """
        self.llm = llm
        self.stance = stance
        self.name = name
        
        # Define the debate prompt for this agent
        self.debate_prompt = PromptTemplate(
            input_variables=["case_analysis", "previous_argument"],
            template="""As a legal expert {stance} for key case classification, analyze this case:

Initial Case Analysis:
{case_analysis}

Previous Argument to Address:
{previous_argument}

Provide your analysis focusing on:

1. Historical Development and Context:
- How does this case relate to previous legal developments?
- What gaps or uncertainties does it address?

2. Legal Framework Development:
- How does this case develop or clarify existing principles?
- What new understanding or guidance does it provide?

3. Contemporary Application:
- How does this case address modern challenges?
- What practical guidance does it offer for current issues?

4. Broader Impact:
- How might this case influence future decisions?
- What aspects make it significant beyond its immediate context?

Support your analysis with specific evidence from the case text.
Focus on substantive developments rather than superficial changes."""
        )

    def analyze(self, case_analysis: str, previous_argument: str = "") -> str:
        """Generate an analysis based on the case and any previous arguments."""
        return self.llm.invoke(
            self.debate_prompt.format(
                stance=self.stance,
                case_analysis=case_analysis,
                previous_argument=previous_argument
            )
        )

# Now, let's create our main debate system
class EnhancedDebateSystem:
    def __init__(self, llm: BaseLLM):
        """Initialize the debate system with necessary components.
        
        Args:
            llm: The language model to use for analysis
        """
        self.llm = llm
        
        # Create our expert agents
        self.advocate = LegalExpertAgent(llm, "advocating", "Advocate")
        self.critic = LegalExpertAgent(llm, "critiquing", "Critic")
        
        # Define our initial analysis prompt
        self.analysis_prompt = PromptTemplate(
            input_variables=["case_facts"],
            template="""As an ECHR legal expert, analyze this case comprehensively:

Case Facts:
{case_facts}

Provide a structured analysis covering:

1. Historical Context and Legal Background:
- What was the state of the law before this case?
- What challenges or uncertainties existed?
- What previous cases or principles are relevant?

2. Core Legal Issues and Development:
- What are the main legal questions addressed?
- How does the case approach these issues?
- What principles or standards does it develop or clarify?

3. Contemporary Relevance:
- How does this case address modern challenges?
- What practical guidance does it provide?
- How does it bridge traditional principles with current needs?

4. Potential Impact:
- How might this influence future cases?
- What broader implications does it have?
- How might other courts use this reasoning?

Support each point with specific evidence from the case text."""
        )
        
        # Define our controller prompt for final decision
        self.controller_prompt = PromptTemplate(
            input_variables=["case_analysis", "debate_history"],
            template="""As a senior ECHR expert, evaluate whether this case should be classified as a key case.

Initial Analysis:
{case_analysis}

Debate Arguments:
{debate_history}

Evaluate the case's significance considering:

1. Legal Development:
- Does it provide important new understanding of legal principles?
- Does it clarify existing principles in significant ways?
- Does it address important gaps or uncertainties?

2. Practical Impact:
- Does it provide valuable guidance for courts?
- Does it help resolve contemporary challenges?
- Does it influence how similar cases should be handled?

3. Broader Significance:
- Does it have implications beyond its immediate context?
- Does it contribute to the development of ECHR jurisprudence?
- Does it provide frameworks for handling similar issues?

Provide your detailed classification decision with specific evidence supporting your conclusions."""
        )

    def perform_initial_analysis(self, case_facts: str) -> str:
        """Conduct initial analysis of the case facts."""
        return self.llm.invoke(
            self.analysis_prompt.format(case_facts=case_facts)
        )

    def conduct_debate(self, initial_analysis: str) -> List[Tuple[str, str]]:
        """Run the debate between advocate and critic agents."""
        debate_history = []
        current_argument = ""

        # Conduct two rounds of debate
        for _ in range(2):
            # Advocate's turn
            advocate_response = self.advocate.analyze(initial_analysis, current_argument)
            debate_history.append(("Advocate", advocate_response))
            current_argument = advocate_response
            
            # Critic's turn
            critic_response = self.critic.analyze(initial_analysis, current_argument)
            debate_history.append(("Critic", critic_response))
            current_argument = critic_response

        return debate_history

    def make_final_decision(self, initial_analysis: str, debate_history: List[Tuple[str, str]]) -> Dict:
        """Generate final decision based on analysis and debate."""
        # Format debate history for the controller
        formatted_debate = "\n\n".join([
            f"{role}:\n{argument}" for role, argument in debate_history
        ])
        
        # Get controller's decision
        final_decision = self.llm.invoke(
            self.controller_prompt.format(
                case_analysis=initial_analysis,
                debate_history=formatted_debate
            )
        )
        
        # Return comprehensive results
        return {
            "is_key_case": "should be classified as a key case" in final_decision,
            "initial_analysis": initial_analysis,
            "debate_history": formatted_debate,
            "final_decision": final_decision
        }

    def evaluate_case(self, case_facts: str) -> Dict:
        """Main method to run the entire analysis process."""
        initial_analysis = self.perform_initial_analysis(case_facts)
        debate_history = self.conduct_debate(initial_analysis)
        return self.make_final_decision(initial_analysis, debate_history)

# Example usage:
def analyze_case(llm: BaseLLM, case_facts: str) -> Dict:
    """Analyze a case using the enhanced debate system.
    
    Args:
        llm: Language model to use
        case_facts: String containing the case details
        
    Returns:
        Dictionary containing analysis results
    """
    system = EnhancedDebateSystem(llm)
    return system.evaluate_case(case_facts)

In [20]:
from langchain.llms import OpenAI  # or your preferred LLM
from langchain.chat_models import ChatOpenAI
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

# Initialize your language model
#llm = OpenAI()
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,  # Lower temperature for more consistent legal reasoning
)

In [21]:
kc_input_dir = '../../kc_classification_data/125_kc_2018_2024/'
not_kc_input_dir = '../../kc_classification_data/125_notkc_2018_2024/'

kc_input_cases = os.listdir(kc_input_dir)
not_kc_input_cases = os.listdir(not_kc_input_dir)

In [22]:
from utils import load_json 

kcase_path = os.path.join(kc_input_dir, kc_input_cases[0])
kcase_data = load_json(kcase_path)
kcase_facts = kcase_data['case_text']['THE FACTS']

nkcase_path = os.path.join(not_kc_input_dir, not_kc_input_cases[0])
nkcase_data = load_json(nkcase_path)
nkcase_facts = nkcase_data['facts']

In [38]:
nkcase_path

'../../kc_classification_data/125_notkc_2018_2024/001-186776.json'

In [26]:
# Analyze a case
case_facts = kcase_facts
results = analyze_case(llm, case_facts)


# Access the results
print(f"Key Case Classification: {results['is_key_case']}")
print(f"Confidence: {results['confidence']}")
print(f"Reasoning: {results['reasoning']}")

Key Case Classification: False


KeyError: 'confidence'

In [27]:
results

{'is_key_case': False,
 'initial_analysis': AIMessage(content='### 1. Historical Context and Legal Background:\n\n**State of the Law Before the Case:**\nBefore this case, the legal framework in Austria regarding freedom of expression and religious peace was primarily governed by Articles 188 and 283 of the Austrian Criminal Code. Article 188 criminalized the disparagement of religious doctrines, while Article 283 addressed incitement to hatred. The European Convention on Human Rights (ECHR), particularly Articles 9 and 10, provided a broader context for balancing freedom of expression with the protection of religious beliefs.\n\n**Challenges or Uncertainties:**\nThe primary challenge was balancing the right to freedom of expression with the need to protect religious peace and prevent hate speech. The interpretation of what constitutes "disparagement" or "incitement" was often subjective, leading to uncertainties in application. The case also highlighted the difficulty in distinguishing

In [29]:
print(results['final_decision'].content)

To evaluate whether this case should be classified as a key case, we need to consider its significance across three main dimensions: legal development, practical impact, and broader significance. Here's a detailed analysis based on the provided information:

### 1. Legal Development

**New Understanding of Legal Principles:**
- The case provides a nuanced understanding of the distinction between factual statements and value judgments, particularly in the context of religious discourse. This is crucial because it helps delineate the boundaries of permissible speech under Article 10 of the ECHR.

**Clarification of Existing Principles:**
- It clarifies the application of Articles 188 and 283 of the Austrian Criminal Code, particularly what constitutes "disparagement" or "incitement." This clarification is significant as it addresses the subjective nature of these terms and provides a clearer framework for their interpretation.

**Addressing Gaps or Uncertainties:**
- The case addresses t

In [33]:
print(results['initial_analysis'].content)

### 1. Historical Context and Legal Background:

**State of the Law Before the Case:**
Before this case, the legal framework in Austria regarding freedom of expression and religious peace was primarily governed by Articles 188 and 283 of the Austrian Criminal Code. Article 188 criminalized the disparagement of religious doctrines, while Article 283 addressed incitement to hatred. The European Convention on Human Rights (ECHR), particularly Articles 9 and 10, provided a broader context for balancing freedom of expression with the protection of religious beliefs.

**Challenges or Uncertainties:**
The primary challenge was balancing the right to freedom of expression with the need to protect religious peace and prevent hate speech. The interpretation of what constitutes "disparagement" or "incitement" was often subjective, leading to uncertainties in application. The case also highlighted the difficulty in distinguishing between legitimate criticism of religious practices and offensive at

In [10]:
print(kcase_facts)

I.  THE CIRCUMSTANCES OF THE CASE
6.  The applicant was born in 1971 and lives in Vienna.
7.  From January 2008 she held several seminars entitled “Basic Information on Islam” (Grundlagen des Islams) at the right-wing Freedom Party Education Institute (Bildungsinstitut der Freiheitlichen Partei Österreichs). The seminars were not only open to members of the Freedom Party or invited guests, but were also publicly advertised on its website. In addition, the head of the Freedom Party, H.-C.S., had distributed a leaflet specifically aimed at young voters, advertising them as “top seminars” in the framework of a “free education package”. The applicant had not been involved in the selection of participants.
8.  Two of the seminars were held on 15 October and 12 November 2009 respectively, with around thirty participants at each. One of the participants was an undercover journalist working for a weekly journal, N.
9.  At the journal’s request, a preliminary investigation was instituted agains

In [34]:
# Analyze a case
case_facts = nkcase_facts
results = analyze_case(llm, case_facts)


# Access the results
print(f"Key Case Classification: {results['is_key_case']}")
print(f"Confidence: {results['confidence']}")
print(f"Reasoning: {results['reasoning']}")

Key Case Classification: False


KeyError: 'confidence'

In [35]:
print(results['final_decision'].content)

To evaluate whether this case should be classified as a key case, we need to consider its significance across three main dimensions: legal development, practical impact, and broader significance. Here's a detailed analysis based on the provided information:

### 1. Legal Development

**New Understanding of Legal Principles:**
- The case provides a new understanding by reclassifying acts from the Romanian Revolution of December 1989 as crimes against humanity. This reclassification is significant because it extends the scope of legal accountability and ensures that such acts are not subject to statute limitations. This is a crucial development in international human rights law, as it emphasizes the gravity of state-sponsored violence and the need for a comprehensive legal response.

**Clarification of Existing Principles:**
- The case clarifies the principle that serious human rights violations require thorough and effective investigations, regardless of the time elapsed. This aligns wi

In [36]:
print(results['debate_history'])

Advocate:
content="### Analysis of the Case\n\n#### 1. Historical Development and Context:\n\n**Relation to Previous Legal Developments:**\nThis case is deeply rooted in the historical context of the Romanian Revolution of December 1989, a pivotal moment that led to the fall of the communist regime. Prior to this case, the legal landscape was marked by protracted investigations into the violent suppression of anti-communist protests. The case builds on previous legal developments, particularly the European Court of Human Rights (ECHR) cases like Association 21 December 1989 and Others v. Romania and Mocanu and Others v. Romania, which emphasized the state's duty to conduct effective investigations into the events of 1989.\n\n**Gaps or Uncertainties Addressed:**\nThe case addresses significant gaps in the legal treatment of the 1989 events, particularly concerning the statute of limitations. It confronts the uncertainty about whether these acts could be classified as crimes against huma

In [37]:
print(nkcase_facts)

I. THE CIRCUMSTANCES OF THE CASE
4. The applicant was born in 1943 and lives in Bucharest.
5. The facts of the case, as submitted by the parties, refer to the same context and domestic criminal proceedings as those described in the case Association 21 December 1989 and Others v. Romania (nos. 33810/07 and 18817/08, 12-41, 24 May 2011).
6. During the events which led to the fall of the communist regime, on the night of 21/22 December 1989 the applicant s son was killed by gunfire in Bucharest.
7. In 1990 the military prosecutor s office opened several investigations into the December 1989 armed crackdown on the anti-communism demonstrations. A main criminal investigation concerning the use of violence against civilians in Bucharest and other cities was registered with the highest prosecutor s office the military prosecutors section under no. 97/P/1990 (current number 11/P/2014). In the main criminal investigation the applicant raised civil claims and asked that the perpetrators of the o